In [1]:
import csv
import pandas as pd
import script.load_data as ld
import script.statistics as stat
from sklearn import model_selection
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

RANDOM_STATE = 1234

In [2]:
DMM234 = True

In [3]:
# Load and clean data
df = ld.load_data()

FileNotFoundError: [Errno 2] No such file or directory: '../data/testset.csv'

In [ ]:
def deDMM(Xt):
    X = Xt.copy()
    
    X["DMM_"] = np.nan

    for cat in [2, 3, 4]:
        X.loc[X['M1' + str(cat)] == 1, ['DMM_']] = cat - 2
    X = X.drop(columns = ['M12', 'M13', 'M14'])
    return X
    

In [ ]:
if DMM234:
    df = deDMM(df)

In [ ]:
# Dataset split
X_train, X_test, y_train, y_test = model_selection.train_test_split(df.drop('target', axis=1), df['target'], 
                                                                    test_size=0.2, stratify=df['target'], 
                                                                    random_state=RANDOM_STATE)

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
rfc = BalancedRandomForestClassifier(n_estimators=1200, criterion='gini', bootstrap=True, oob_score=True, n_jobs=None, 
                            random_state=RANDOM_STATE, warm_start=False, class_weight='balanced')

# Train the model on training data
rfc.fit(SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(X_train), y_train)

In [ ]:
# Proviamo con XGBoost
import xgboost
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

param_dist = {'n_estimators': 500
              ,'learning_rate': 0.03
              ,'subsample': 1
              ,'max_depth': 6
              ,'colsample_bytree': 0.5
              ,'min_child_weight': 1
             # , 'scale_pos_weight' : 0.22
             }


xgbc = xgboost.XGBClassifier(objective = 'binary:logistic', **param_dist)
xgbc.fit(X_train, y_train)

In [ ]:
missclassified = np.bitwise_xor(xgbc.predict(X_test), y_test.to_numpy())

In [ ]:
missclassified = np.array(missclassified, dtype=bool)

In [ ]:
proba = xgbc.predict_proba(X_test)

In [ ]:
proba

In [ ]:
prob_miss = proba[missclassified, 1]

In [ ]:
prob_miss

In [ ]:
plt.hist(prob_miss, bins=100)
plt.show()

In [ ]:
# Proviamo con CatBoost
def deohe(Xt):
    X = Xt.copy()
    
    if DMM234:  
        X[['DMM']] = X[['nuovo cliente']]
        X[['BU']] = X[['nuovo cliente']]
        X[['Prodotto']] = X[['nuovo cliente']]
        for cat in ['M15', 'M16', 'M17', 'Z17']:
            X.loc[X[cat] == 1, ['DMM']] = cat
        X = X.drop(columns = ['M15', 'M16', 'M17', 'Z17'])
    
    else:
        X[['DMM']] = X[['nuovo cliente']]
        X[['BU']] = X[['nuovo cliente']]
        X[['Prodotto']] = X[['nuovo cliente']]
        for cat in ['M12', 'M13', 'M14', 'M15', 'M16', 'M17', 'Z17']:
            X.loc[X[cat] == 1, ['DMM']] = cat
        X = X.drop(columns = ['M12', 'M13', 'M14', 'M15', 'M16', 'M17', 'Z17'])
    
    
    for cat in ['BLS', 'ELS', 'RD',
       'TS']:
        X.loc[X[cat] == 1, ['BU']] = cat
    X = X.drop(columns = ['BLS', 'ELS', 'RD',
       'TS'])
    
    for cat in ['Cessione Contratto Mandatata', 'Cessione Contratto Semplice',
       'Credito', 'Enveloppe loc, opérations Spé',
       'Enveloppe location Transfert', 'Enveloppe location immobilier',
       'Locaz. Finanziaria Immobiliare', 'Locazione Finanziaria',
       'Locazione Operativa', 'Package credito', 'Package locazione',
       'Riacquisto di crediti']:
        X.loc[X[cat] == 1, ['Prodotto']] = cat
    X = X.drop(columns = ['Cessione Contratto Mandatata', 'Cessione Contratto Semplice',
       'Credito', 'Enveloppe loc, opérations Spé',
       'Enveloppe location Transfert', 'Enveloppe location immobilier',
       'Locaz. Finanziaria Immobiliare', 'Locazione Finanziaria',
       'Locazione Operativa', 'Package credito', 'Package locazione',
       'Riacquisto di crediti'])
    
    return X

In [ ]:
from catboost import CatBoostClassifier
grid = {'learning_rate': 0.03,
        'depth': 6,
        'l2_leaf_reg': 7,
        'iterations': 1000,
        'class_weights' : [1, 0.22]
        }
cbc = CatBoostClassifier(loss_function='Logloss', 
                           cat_features = ['DMM', 'BU', 'Prodotto', 'nuovo cliente', 'nuovo gruppo'],
                           **grid)

cbc.fit(deohe(X_train), y_train)

In [ ]:
import numpy as np
from sklearn.base import ClassifierMixin
class MixEstimator(ClassifierMixin):
    def __init__(self, est1, est2, est3):
        self.est1 = est1
        self.est2 = est2
        self.est3 = est3
        self.classes_ = [0,1]
    def predict_proba(self, X):
        return np.mean(np.dstack((self.est1.predict_proba(X), self.est2.predict_proba(deohe(X)), 
                                  self.est3.predict_proba(SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(X)))), 
                       axis = 2)
    
    def predict(self, X):
        return np.logical_and(np.logical_or(self.est2.predict(deohe(X)), 
                            self.est3.predict(SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(X))),
                            self.est1.predict(X))*1

In [ ]:
MixEstimator(xgbc, cbc, rfc).predict(X_test)

In [ ]:
from sklearn.metrics import balanced_accuracy_score
for model, X in [(rfc, SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(X_test)),
                 (xgbc, X_test),
                 (cbc, deohe(X_test)),
                 (MixEstimator(xgbc, cbc, rfc), X_test)]:
    print(model.__class__.__name__)
    print(balanced_accuracy_score(y_test, model.predict(X), adjusted=True))
    

In [ ]:
from sklearn.metrics import plot_confusion_matrix
for model, X in [(rfc, SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(X_test)),
                 (xgbc, X_test),
                 (cbc, deohe(X_test)),
                 (MixEstimator(xgbc, cbc, rfc), X_test)]:
    print(model.__class__.__name__)
    plot_confusion_matrix(model, X, y_test, normalize='true')
    plt.show()

In [ ]:
## ENDS

In [ ]:
# Baseline: Random Forest

# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

# Create our imputer to replace missing values with the mean e.g.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X_train)

# Impute our data, then train
X_train_imp = imp.transform(X_train)

# Instantiate model
rfc = RandomForestClassifier(n_estimators=1200, criterion='gini', max_depth=None, 
                            min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                            max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                            min_impurity_split=None, bootstrap=True, oob_score=True, n_jobs=None, 
                            random_state=RANDOM_STATE, warm_start=False, class_weight=None, 
                            ccp_alpha=0.0, max_samples=None)

# Train the model on training data
rfc.fit(X_train_imp, y_train)

In [ ]:
from sklearn.utils import resample
df_majority = df[df.target==1]
df_minority = df[df.target==0]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=17000,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.target.value_counts()
# 1    49
# 0    49
# Name: balance, dtype: int64

In [ ]:
## ENDS

In [ ]:
# Baseline: Random Forest

# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

# Create our imputer to replace missing values with the mean e.g.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X_train)

# Impute our data, then train
X_train_imp = imp.transform(X_train)

# Instantiate model
rfc = RandomForestClassifier(n_estimators=1200, criterion='gini', max_depth=None, 
                            min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                            max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                            min_impurity_split=None, bootstrap=True, oob_score=True, n_jobs=None, 
                            random_state=RANDOM_STATE, warm_start=False, class_weight=None, 
                            ccp_alpha=0.0, max_samples=None)

# Train the model on training data
rfc.fit(X_train_imp, y_train)

In [ ]:
from sklearn.utils import resample
df_majority = df[df.target==1]
df_minority = df[df.target==0]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=17000,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.target.value_counts()
# 1    49
# 0    49
# Name: balance, dtype: int64

In [ ]:
## ENDS

In [ ]:
# Baseline: Random Forest

# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

# Create our imputer to replace missing values with the mean e.g.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X_train)

# Impute our data, then train
X_train_imp = imp.transform(X_train)

# Instantiate model
rfc = RandomForestClassifier(n_estimators=1200, criterion='gini', max_depth=None, 
                            min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                            max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                            min_impurity_split=None, bootstrap=True, oob_score=True, n_jobs=None, 
                            random_state=RANDOM_STATE, warm_start=False, class_weight=None, 
                            ccp_alpha=0.0, max_samples=None)

# Train the model on training data
rfc.fit(X_train_imp, y_train)

In [ ]:
from sklearn.utils import resample
df_majority = df[df.target==1]
df_minority = df[df.target==0]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=17000,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.target.value_counts()
# 1    49
# 0    49
# Name: balance, dtype: int64

In [ ]:
## ENDS

In [ ]:
# Baseline: Random Forest

# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

# Create our imputer to replace missing values with the mean e.g.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X_train)

# Impute our data, then train
X_train_imp = imp.transform(X_train)

# Instantiate model
rfc = RandomForestClassifier(n_estimators=1200, criterion='gini', max_depth=None, 
                            min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                            max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                            min_impurity_split=None, bootstrap=True, oob_score=True, n_jobs=None, 
                            random_state=RANDOM_STATE, warm_start=False, class_weight=None, 
                            ccp_alpha=0.0, max_samples=None)

# Train the model on training data
rfc.fit(X_train_imp, y_train)

In [ ]:
from sklearn.utils import resample
df_majority = df[df.target==1]
df_minority = df[df.target==0]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=17000,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.target.value_counts()
# 1    49
# 0    49
# Name: balance, dtype: int64

In [ ]:
## ENDS

In [ ]:
# Baseline: Random Forest

# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

# Create our imputer to replace missing values with the mean e.g.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X_train)

# Impute our data, then train
X_train_imp = imp.transform(X_train)

# Instantiate model
rfc = RandomForestClassifier(n_estimators=1200, criterion='gini', max_depth=None, 
                            min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                            max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                            min_impurity_split=None, bootstrap=True, oob_score=True, n_jobs=None, 
                            random_state=RANDOM_STATE, warm_start=False, class_weight=None, 
                            ccp_alpha=0.0, max_samples=None)

# Train the model on training data
rfc.fit(X_train_imp, y_train)

In [ ]:
from sklearn.utils import resample
df_majority = df[df.target==1]
df_minority = df[df.target==0]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=17000,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.target.value_counts()
# 1    49
# 0    49
# Name: balance, dtype: int64

In [ ]:
## ENDS

In [ ]:
# Baseline: Random Forest

# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

# Create our imputer to replace missing values with the mean e.g.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X_train)

# Impute our data, then train
X_train_imp = imp.transform(X_train)

# Instantiate model
rfc = RandomForestClassifier(n_estimators=1200, criterion='gini', max_depth=None, 
                            min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                            max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                            min_impurity_split=None, bootstrap=True, oob_score=True, n_jobs=None, 
                            random_state=RANDOM_STATE, warm_start=False, class_weight=None, 
                            ccp_alpha=0.0, max_samples=None)

# Train the model on training data
rfc.fit(X_train_imp, y_train)

In [ ]:
from sklearn.utils import resample
df_majority = df[df.target==1]
df_minority = df[df.target==0]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=17000,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.target.value_counts()
# 1    49
# 0    49
# Name: balance, dtype: int64

In [ ]:
df = df_downsampled